In [1]:
import requests
from datetime import datetime, timedelta
from pprint import pprint

In [2]:
# 측정소별 실시간 측정정보 조회 (매시 15분 내외 업데이트)
base_url = "http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty"
with open('keys/에어코리아api.txt') as file:
    service_key = file.read()       # 발급받은 에어코리아 API 키 입력

In [3]:
# 웹 요청시 같이 전달될 데이터 = 요청 메시지
params = {
        'serviceKey' : service_key,
        'returnType' : 'JSON',
        'numOfRows' : 30,
        'pageNo' : 1,
        'stationName' : "강변북로", # 측정소이름
        'dataTerm' : 'DAILY',
        'ver' : "1.4" # 오퍼레이션 버전
    }

res = requests.get(url=base_url , params=params)
# print(res.status_code, type(res.text), res.url)
# print()
# print(res.text)

# 응답 데이터 정리
from pprint import pprint  # 구조있는 데이터를 더 편하게 보여줌
data = res.json() # json.loads(res.text)와 같은 기능
data = data['response']['body']['items']
pprint(data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import pandas as pd
station_Name = pd.read_csv('data/대기측정1번.csv', encoding='utf-8')

In [ ]:
station_Name.측정소명

0      송정동(봉명동)
1           영동읍
2           영천동
3           오송읍
4           오창읍
         ...   
120          연동
121         이도동
122         조천읍
123         한림읍
124         화북동
Name: 측정소명, Length: 125, dtype: object

In [ ]:
# 완성코드

def get_air_quality():
    
    # 결과를 저장할 리스트
    results_list = []
    
    # 각 측정소에 대한 데이터 수집
    for index, row in station_Name.iterrows():
        # 웹 요청시 같이 전달될 데이터 = 요청 메시지
        params = {
            'serviceKey' : service_key,
            'returnType' : 'JSON',
            'numOfRows' : 30,
            'pageNo' : 1,
            'stationName' : row['측정소명'], # 측정소이름
            'dataTerm' : 'DAILY',
            'ver' : "1.4" # 오퍼레이션 버전
        }

        res = requests.get(url=base_url , params=params)

        # 응답 데이터 정리
        from pprint import pprint  # 구조있는 데이터를 더 편하게 보여줌
        data = res.json() # json.loads(res.text)와 같은 기능
        data = data['response']['body']['items']
        
        # 데이터를 날짜와 시간에 대한 기준으로 정렬
        sorted_data = sorted(data, key=lambda x: x['dataTime'], reverse=True)

        # 최신 데이터 선택
        latest_data = sorted_data[0]
        
        # 최종 데이터 생성
        results = {
            '날짜': latest_data['dataTime'],
            '이름': latest_data['stationName'],
            '측정망 정보': latest_data['mangName'],
            '아황산가스 농도': latest_data['so2Value'] + "ppm",
            '일산화탄소 농도': latest_data['coValue'] + "ppm",
            '오존 농도': latest_data['o3Value'] + "ppm",
            '이산화질소 농도': latest_data['no2Value'] + "ppm",
            '미세먼지(PM10) 농도': latest_data['pm10Value'] + "ug/m³",
            '초미세먼지(PM2.5) 농도': latest_data['pm25Value'] + "ug/m³"
        }
        
        # 결과 리스트에 추가
        results_list.append(results)
            
        # pprint(results)
        return results_list
    
# 테스트 코드
if __name__ == "__main__":
    print(get_air_quality())

[{'날짜': '2023-11-25 12:00', '이름': '송정동(봉명동)', '측정망 정보': '도시대기', '아황산가스 농도': '0.002ppm', '일산화탄소 농도': '0.4ppm', '오존 농도': '0.026ppm', '이산화질소 농도': '0.010ppm', '미세먼지(PM10) 농도': '17ug/m³', '초미세먼지(PM2.5) 농도': '11ug/m³'}]


In [ ]:
import requests
import pandas as pd

# 예제 데이터프레임 생성
station_Name = pd.read_csv('data/대기측정1번.csv', encoding='utf-8')

# 완성코드
def get_air_quality():
    # 결과를 저장할 리스트
    results_list = []
    
    # 각 측정소에 대한 데이터 수집
    for index, row in station_Name.iterrows():
        # 웹 요청시 같이 전달될 데이터 = 요청 메시지
        params = {
            'serviceKey' : service_key,
            'returnType' : 'JSON',
            'numOfRows' : 30,
            'pageNo' : 1,
            'stationName' : row['측정소명'],  # 각 행의 측정소명을 선택
            'dataTerm' : 'DAILY',
            'ver' : "1.4"  # 오퍼레이션 버전
        }

        res = requests.get(url=base_url , params=params)

        # 응답 데이터 정리
        data = res.json()['response']['body']['items']
        
        # 데이터가 존재하면 처리
        if data:
            # 데이터를 날짜와 시간에 대한 기준으로 정렬
            sorted_data = sorted(data, key=lambda x: x['dataTime'], reverse=True)

            # 최신 데이터 선택
            latest_data = sorted_data[0]

            # 최종 데이터 생성
            result = {
                '날짜': latest_data['dataTime'],
                '이름': latest_data['stationName'],
                '측정망 정보': latest_data['mangName'],
                '아황산가스 농도': latest_data['so2Value'] + "ppm",
                '일산화탄소 농도': latest_data['coValue'] + "ppm",
                '오존 농도': latest_data['o3Value'] + "ppm",
                '이산화질소 농도': latest_data['no2Value'] + "ppm",
                '미세먼지(PM10) 농도': latest_data['pm10Value'] + "ug/m³",
                '초미세먼지(PM2.5) 농도': latest_data['pm25Value'] + "ug/m³"
            }

            # 결과 리스트에 추가
            results_list.append(result)
    
    return results_list

# 테스트 코드
if __name__ == "__main__":
    results = get_air_quality()
    for result in results:
        print(result)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
print(res.text)

{"response":{"body":{"totalCount":23,"items":[{"pm25Grade1h":null,"pm10Value24":"14","so2Value":"0.002","pm10Grade1h":"1","o3Grade":"1","pm10Value":"9","pm25Flag":"통신장애","khaiGrade":"1","pm25Value":"-","no2Flag":null,"mangName":"도시대기","stationName":"서대문구","no2Value":"0.014","so2Grade":"1","stationCode":"111191","coFlag":null,"khaiValue":"42","coValue":"0.3","pm10Flag":null,"no2Grade":"1","pm25Value24":"10","o3Flag":null,"pm25Grade":"1","so2Flag":null,"coGrade":"1","dataTime":"2023-11-25 12:00","pm10Grade":"1","o3Value":"0.025"},{"pm25Grade1h":"1","pm10Value24":"15","so2Value":"0.002","pm10Grade1h":"1","o3Grade":"1","pm10Value":"11","pm25Flag":null,"khaiGrade":"1","pm25Value":"11","no2Flag":null,"mangName":"도시대기","stationName":"서대문구","no2Value":"0.019","so2Grade":"1","stationCode":"111191","coFlag":null,"khaiValue":"33","coValue":"0.4","pm10Flag":null,"no2Grade":"1","pm25Value24":"10","o3Flag":null,"pm25Grade":"1","so2Flag":null,"coGrade":"1","dataTime":"2023-11-25 11:00","pm10Grade":"1